In [1]:
import numpy as np
import math
Q = 3
N = 2**Q-1
h = 1/N
h2 = h * h
err = 10
err_p = 1e-3

In [9]:
def geterr(yn, y):
    return sum((yn-y)**2)**0.5
def getf(y):
    fn = np.ones(N)
    fn[0] = 1
    fn[N-1] = 1
    for i in range(1,N-1):
        fn[i] = (math.exp(y[i])+(math.exp(y[i+1])- 2*math.exp(y[i]) + math.exp(y[i-1]))/12)
    return fn
def iter(M, y):
    data = M.copy()
    yn = y.copy()
    vf = getf(y)
    nn = N
    low = 2
    stride = 1
    while(nn > 1):
        for i in range(low-1, N, stride * 2):
            alpha = -data[i][i-1] / data[i - stride][i-stride]
            gamma = -data[i][i+1] / data[i + stride][i+stride]
            if (i - stride >= 1):
                data[i][i-1] = alpha * data[i - stride][i-stride-1]
            data[i][i] = alpha * data[i - stride][i-stride+1] + data[i][i] + gamma * data[i + stride][i+stride-1]
            if (i + stride < N - 1):
                data[i][i+1] = gamma * data[i + stride][i+stride+1]
            vf[i] = alpha * vf[i - stride] + vf[i] + gamma * vf[i + stride]
        nn = nn//2
        low = low * 2
        stride = stride * 2

    yn[N//2] = vf[N//2] / data[N//2][N//2];
    stride = stride // 2
    while (stride >= 1):
        for i in range(stride - 1, N, stride * 2):
            yn[i] = (vf[i]
            - (data[i][i-1] * yn[i - stride] if i- stride > 0 else 0.0)
            - (data[i][i+1] * yn[i + stride] if i +stride < N  else 0.0)
            ) / data[i][i]
        stride = stride // 2
    return yn

def init_M():
    M = np.zeros((N, N))
    M[0][0] = 1
    M[N-1][N-1] = 1
    for i in range(1,N-1):
        M[i][i] = -2 /h2
        if (i > 0):
            M[i][i-1] = 1 /h2
        if (i < N - 1):
            M[i][i+1] = 1 /h2
    return M

def prep():
    return np.ones(N), np.ones(N), getf(np.ones(N)), init_M()

In [10]:
y,yn,f,M = prep()

for j in range(10):
    yn = iter(M, y)
    err = geterr(yn, y)
    y = yn.copy()

print(y)

[1.         0.88169722 0.81292288 0.79034587 0.81292288 0.88169722
 1.        ]


In [11]:
y,yn,f,M = prep()

for j in range(10):
    yn = np.linalg.solve(M, f)
    f = getf(yn)
    err = geterr(yn, y)
    y = yn.copy()

print(y)

[1.         0.88169722 0.81292288 0.79034587 0.81292288 0.88169722
 1.        ]


In [12]:
y,yn,f,M = prep()
print(iter(M, y))
print(np.linalg.solve(M, getf(y)))

[1.         0.86131215 0.77809944 0.75036187 0.77809944 0.86131215
 1.        ]
[1.         0.86131215 0.77809944 0.75036187 0.77809944 0.86131215
 1.        ]
